This script takes Texas Education Agency data about school district demographics and disciplinary actions, and puts them together in one GeoJSON file for the Texas Appleseed "School to Prison Pipeline" map. See http://www.texasdisciplinelab.org/

To use the script, follow these instructions:

1. For every year that you want to cover, download all 20 of the region files from http://rptsvr1.tea.texas.gov/adhocrpt/Disciplinary_Data_Products/Download_Region_Districts.html and put them in the directory '../data/from_agency/by_region/'

2. For every year that you want to cover, download the "District and Charter Detail Data" Snapshot Data File (comma-delimited *.dat)" from https://rptsvr1.tea.texas.gov/perfreport/snapshot/download.html. The website automatically delivers these files with the same filename: district.dat. You will need to rename them to have different names by adding the year after "district". For instance, "district2016.dat"

3. This script needs a GeoJSON file of district shapes. Make sure it can find that file at '../geojson/base_districts.geojson'


In [26]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 999

first_year = 2006 # the year 2006 is the first year on the TEA site
last_year = 2016


def formatDF(apple, year_col):
    
    # Removes rows and columns not needed for the map
    
    apple = apple.drop(["AGGREGATION LEVEL","REGION","DISTNAME"], axis = 1)
    
    # Adding a totals row by adding up actions against special ed students and non-special ed students
    
    non_special = {"D06": ("D05","D-EXPULSION ACTIONS"), 
                   "D09": ("D08","E-DAEP PLACEMENTS"), 
                   "D12": ("D11", "F-OUT OF SCHOOL SUSPENSIONS"), 
                   "D15": ("D14", "G-IN SCHOOL SUSPENSIONS")}
    
    # This will be inefficient because it makes a dict first instead of staying in pandas
    
    all_actions = []
    
    # if it was a .csv, the headers would be ["DISTRICT", "SECTION", "HEADING", "HEADING NAME", year_col]
    
    for d in apple.DISTRICT.unique():
        for key in non_special:
            section = non_special[key][1]
            try: 
                a = apple.loc[(apple["DISTRICT"] == d) & (apple["HEADING"] == key), year_col].values[0]
            except IndexError:
                a = 0
            try:
                b = apple.loc[(apple["DISTRICT"] == d) & (apple["HEADING"] == non_special[key][0]), year_col].values[0]
            except IndexError:
                b = 0
            if a < 0: # in case of dummy values like -999
                a = 1
            if b < 0:
                b = 1
            total = a + b
            all_actions.append({"DISTRICT": d, "SECTION": section, "HEADING": key, "HEADING NAME": "TOTAL", 
                               year_col: total})
    
    new = pd.DataFrame(all_actions)        

    # Keeping only the rows that categorize students by protected class.
    patternIn = 'WHITE|BLACK OR AFRICAN AMERICAN|AMERICAN INDIAN OR ALASKA NAT|HISPANIC|NATIVE HAWAIIAN|ASIAN|TWO OR MORE RACES|SPEC. ED|ECO. DISAD|ECO DISAD.|TOTAL'
    apple = apple[apple["HEADING NAME"].str.contains(patternIn)]
    
    # Getting rid of rows that count students instead of incidents, or non-disadvantaged kids.
    patternOut = 'SPEC. ED. STUDENTS| SPEC. ED. EXPULSIONS TO JJAEP|ECO DISAD. STUDENTS|ECO. DISAD. STUDENTS|AT RISK|NON AT|UNKNOWN AT|NON SPEC. ED.|NON ECO DISAD.|NON ECO. DISAD.'
    apple = apple[apple["HEADING NAME"].str.contains(patternOut) == False]

    # Consolidating some of the descriptors into broader categories
    appleReplace = {year_col:
                        {-99999999: 1, -999999: 1, -999: 1},
                    "SECTION": {
                        'M-ECO\. DISADV\. JJAEP PLACEMENTS|H-SPEC\. ED\. JJAEP EXPULSIONS': 'C-JJAEP EXPULSIONS',
                        'N-ECO\. DISADV\. EXPULSIONS|I-SPEC\. ED\. EXPULSIONS': 'D-EXPULSION ACTIONS',
                        'O-ECO\. DISADV\. DAEP PLACEMENTS|J-SPEC\. ED\. DAEP PLACEMENTS': 'E-DAEP PLACEMENTS',
                        'P-ECO\. DISADV\. OUT OF SCHOOL SUS.|K-SPEC\. ED\. OUT OF SCHOOL SUS\.': 'F-OUT OF SCHOOL SUSPENSIONS',
                        'Q-ECO\. DISADV\. IN SCHOOL SUS\.|L-SPEC\. ED\. IN SCHOOL SUS\.': 'G-IN SCHOOL SUSPENSIONS'},
                    "HEADING NAME": {'SPEC\. ED.*$': 'Special Education',
                                     'ECO?. DISAD.*$': 'Economic Disadvantage'}
                    }

    apple = apple.replace(to_replace=appleReplace, regex=True)

    # Delete rows appearing to double-count the same expulsions.
    apple = apple[apple["SECTION"].str.contains("JJAEP EXPULSIONS|DISCIPLINE ACTION COUNTS") == False]
    
    apple = pd.concat([apple,new])
                                          
    return apple

def getYear(year):
    year_col = "YR{}".format(str(year)[-2:])
    apple_path = '../data/from_agency/by_region/REGION_{}_DISTRICT_summary_{}.csv'
    one_year = [pd.read_csv(apple_path.format(str(region).zfill(2),str(year)[-2:]), dtype = {year_col: int})
            for region in range(1,21)]
    a = pd.concat(one_year)
    
    # a = a.set_index(["DISTRICT","HEADING"] )
    # a = a[~a.index.duplicated(keep='last')]  # a single row was causing a non-unique multiindex error 
    
    a = formatDF(a, year_col)
    return a

apple = getYear(first_year)

In [27]:
apple.index.get_level_values(0)

Int64Index([  20,   21,   22,   23,   24,   26,   27,   29,   30,   32,
            ...
            4898, 4899, 4900, 4901, 4902, 4903, 4904, 4905, 4906, 4907],
           dtype='int64', length=17878)

In [28]:
apple.loc[(apple["DISTRICT"] == 4901) & (apple["HEADING"] == "C18"),"YR06"].values[0]

81

In [29]:
# df.name.unique()

apple.DISTRICT.unique()

array([ 31901, 108902, 108806, ...,  15808,  15826,  15806])

In [30]:
apple[-10:]


,DISTRICT,HEADING,HEADING NAME,SECTION,YR06
4898,247906,D12,TOTAL,\tF-OUT OF SCHOOL SUSPENSIONS,7
4899,247906,D15,TOTAL,G-IN SCHOOL SUSPENSIONS,347
4900,232904,D06,TOTAL,D-EXPULSION ACTIONS,0
4901,232904,D09,TOTAL,E-DAEP PLACEMENTS,0
4902,232904,D12,TOTAL,\tF-OUT OF SCHOOL SUSPENSIONS,0
4903,232904,D15,TOTAL,G-IN SCHOOL SUSPENSIONS,2
4904,232903,D06,TOTAL,D-EXPULSION ACTIONS,18
4905,232903,D09,TOTAL,E-DAEP PLACEMENTS,230
4906,232903,D12,TOTAL,\tF-OUT OF SCHOOL SUSPENSIONS,560
4907,232903,D15,TOTAL,G-IN SCHOOL SUSPENSIONS,1706


In [4]:
for year in range(first_year + 1, last_year + 1):
    new_year = getYear(year)
    new_year = new_year.drop(["HEADING NAME","SECTION"], axis = 1)
    apple = pd.merge(apple, new_year, how='outer', on = ["DISTRICT", "HEADING"],
                     suffixes = (str(year-1), str(year))).fillna(value = 0) # left_index = True, right_index=True, 

In [8]:
apple[1050:1065]

,DISTRICT,HEADING,HEADING NAME,SECTION,YR06
58,158901,C08,HISPANIC,D-EXPULSION ACTIONS,5
59,158901,C10,WHITE,D-EXPULSION ACTIONS,1
61,158901,C12,ASIAN,E-DAEP PLACEMENTS,1
62,158901,C13,HISPANIC,E-DAEP PLACEMENTS,40
63,158901,C15,WHITE,E-DAEP PLACEMENTS,31
65,158901,C17,ASIAN,F-OUT OF SCHOOL SUSPENSIONS,1
66,158901,C18,HISPANIC,F-OUT OF SCHOOL SUSPENSIONS,517
67,158901,C20,WHITE,F-OUT OF SCHOOL SUSPENSIONS,154
69,158901,C22,ASIAN,G-IN SCHOOL SUSPENSIONS,8
70,158901,C23,HISPANIC,G-IN SCHOOL SUSPENSIONS,1714


In [6]:
def populations(districtPath):
    district = pd.read_csv(districtPath, index_col="DISTRICT")

    district = district.rename(columns = {"SNAPDIST": 'DISTNAME'})
    
    sometimes_missing = [ 'DPETINDP', 'DPETASIP', 'DPETPCIP', 'DPETTWOP']
    
    for c in sometimes_missing:
        if c not in district.columns:
            district[c] = np.nan
    
    # deleting redundant columns
    
    district = district[['DISTNAME', 'REGION', 'DPETALLC', 
                         'DPETBLAP', 'DPETHISP', 'DPETWHIP', 'DPETINDP',
                         'DPETASIP', 'DPETPCIP', 'DPETTWOP', 'DPETECOP', 
                         'DPETSPEP']] # 'DISTRICT' not listed because it's the index

    groups = {'AMERICAN INDIAN OR ALASKA NAT': 'DPETINDP',
              'ASIAN': 'DPETASIP',
              'BLACK OR AFRICAN AMERICAN': 'DPETBLAP',
              'Economic Disadvantage': 'DPETECOP',
              'HISPANIC/LATINO': 'DPETHISP',
              'NATIVE HAWAIIAN/OTHER PACIFIC': 'DPETPCIP',
              'Special Education': 'DPETSPEP',
              'TWO OR MORE RACES': 'DPETTWOP',
              'WHITE': 'DPETWHIP'}

    # Using percentages to get counts of students in each group.

    for key in groups:
        try: 
            district[key] = district["DPETALLC"] * district[groups[key]] // 100
        except ValueError:
            district[key] = np.nan
    return district

    """    
    district = district.astype({'BLACK OR AFRICAN AMERICAN': int, 
                                    'HISPANIC/LATINO': int, 
                                    'WHITE': int,
                                    'AMERICAN INDIAN OR ALASKA NAT': int, 
                                    'ASIAN': int,
                                    'NATIVE HAWAIIAN/OTHER PACIFIC': int,
                                    'TWO OR MORE RACES': int, 
                                    'Economic Disadvantage': int, 
                                    'Special Education': int})
    """

    


"""def getRacePop(df, row):
    return df.ix[row["DISTRICT"]][row["HEADING NAME"]]"""


def getRatio(distPop, racePop, all_punishments, group_punishments):
    # Calculating ratio of punishments for the demographic group compared to the punishments for the student population
    # as a whole. For instance, "0.505" in the disparity column indicates the group got the punishment 50.5% as often
    # as average for the student population.

    """
    >>> getRatio(200, 20, 20, 10)
    4.0
    >>> getRatio(200, 20, 20, 2)
    0.0
    >>> print(getRatio(200, 0, 20, 0))
    None
    """

    if max(racePop, group_punishments) == 0 or None:
        return None
    elif all_punishments == 0 or None:
        return 0
    else:
        disparity = (group_punishments / (max(all_punishments, group_punishments))
                     / (max(racePop, group_punishments) / distPop)) - 1
        disparity = Decimal(disparity)
        disparity = disparity.quantize(Decimal('0.01'))
    return float(disparity)

In [7]:
def impossible(distPop, racePop, all_punishments, group_punishments):
    # The "RecordError" column flags implausible data entries. Some of them could still be true if school administrators
    # applied different standards different standards to determine which students belong to which demographic group.
    # Or some could be the result of students not being counted because of the time they moved in and out of district.

    """
    >>> print(impossible(5, 20, 20, 10))
    True
    >>> impossible(20, 0, 20, 0)
    False
    """

    impossible = False
    if group_punishments > all_punishments or racePop > distPop:
        impossible = True
    if racePop == 0 and group_punishments > 0:
        impossible = True
    return impossible


def getFisher(distPop, racePop, all_punishments, group_punishments):
    # I don't know if this is a valid way to report the Fisher's exact test statistic, but the idea is that if getFisher returns a
    # positive number over .95, there's a 95% chance that the group's better-than-average treatment is not due to chance.
    # If it returns a number under -.95, there's a 95% chance that the group's worse-than-average treatment is not due to chance.
    # I think it should be easier to create a color scale to show the scores on a map this way.

    # The getFisher function assumes wrongly that everyone can have only one punishment (of each type). If the number of
    # punishments exceeds the number of kids, it reduces the number of punishments (and assumes wrongly that every
    # kid has been punished) But maybe the results are still close enough to correct to use for scaling?

    """
    >>> getFisher(20, 5, 20, 10)
    0.904604
    >>> getFisher(20, 0, 20, 0)
    None
    """

    if max(racePop, group_punishments) == 0 or None:
        return None
    elif all_punishments == 0 or None:
        return 0
    else:
        oddsratio, pvalueG = stats.fisher_exact([[racePop, max(distPop - racePop, 0)],
                                                 [group_punishments, max(all_punishments - group_punishments, 0)]],
                                                alternative='greater')
        oddsratio, pvalueL = stats.fisher_exact([[racePop, max(distPop - racePop, 0)],
                                                 [group_punishments, max(all_punishments - group_punishments, 0)]],
                                                alternative='less')
        if pvalueL < pvalueG:
            pv = 1 - pvalueL
        else:
            pv = pvalueG - 1
        pv = Decimal(pv)
        pv = pv.quantize(Decimal('0.000001'))
    return float(pv)

In [8]:
import json
with open("../geojson/base_districts.geojson") as json_data:
    district_map = json.load(json_data)
    json_data.close()

In [9]:
shapeIDs = set()

for shape in district_map["features"]:
    shape["id"] = shape["properties"]["DISTRICT_N"]
    assert shape["id"] not in shapeIDs, "id already in list: %r" % shape["id"]
    shapeIDs.add(shape["id"])


In [10]:
year = "2006"  # put the year to use to label the output file (for 2015-16, I use "2016")
districtPath = '../data/from_agency/districts/district{}.dat'.format(year)  # put the path to the district demographics directory here

district = populations(districtPath)

In [11]:
district.head()

,DISTNAME,REGION,DPETALLC,DPETBLAP,DPETHISP,DPETWHIP,DPETINDP,DPETASIP,DPETPCIP,DPETTWOP,DPETECOP,DPETSPEP,AMERICAN INDIAN OR ALASKA NAT,ASIAN,BLACK OR AFRICAN AMERICAN,Economic Disadvantage,HISPANIC/LATINO,NATIVE HAWAIIAN/OTHER PACIFIC,Special Education,TWO OR MORE RACES,WHITE
DISTRICT,,,,,,,,,,,,,,,,,,,,,
1902,Cayuga,7,582,14,4,81,NaN,NaN,NaN,NaN,40.2,16,NaN,NaN,81,233.0,23,NaN,93,NaN,471
1903,Elkhart,7,1294,9,5,86,NaN,NaN,NaN,NaN,47.6,17,NaN,NaN,116,615.0,64,NaN,219,NaN,1112
1904,Frankston,7,797,14,7,79,NaN,NaN,NaN,NaN,50.2,14,NaN,NaN,111,400.0,55,NaN,111,NaN,629
1906,Neches,7,329,8,4,87,NaN,NaN,NaN,NaN,38.0,21,NaN,NaN,26,125.0,13,NaN,69,NaN,286
1907,Palestine,7,3372,30,30,40,NaN,NaN,NaN,NaN,63.5,11,NaN,NaN,1011,2141.0,1011,NaN,370,NaN,1348


In [19]:
# df[df['A'] > 0]

q = district_map["features"][900]["properties"]["DISTRICT_N"]

district.loc[q]

# district[district["DISTRICT"] == 167903]

DISTNAME                         Randolph Field
REGION                                       20
DPETALLC                                   1063
DPETBLAP                                     23
DPETHISP                                     15
DPETWHIP                                     58
DPETINDP                                    NaN
DPETASIP                                    NaN
DPETPCIP                                    NaN
DPETTWOP                                    NaN
DPETECOP                                   14.6
DPETSPEP                                     11
AMERICAN INDIAN OR ALASKA NAT               NaN
ASIAN                                       NaN
BLACK OR AFRICAN AMERICAN                   244
Economic Disadvantage                       155
HISPANIC/LATINO                             159
NATIVE HAWAIIAN/OTHER PACIFIC               NaN
Special Education                           116
TWO OR MORE RACES                           NaN
WHITE                                   

In [25]:
pop_stats = ["DPETALLC","DPETBLAP","DPETHISP","DPETWHIP","DPETINDP","DPETASIP","DPETPCIP",
             "DPETTWOP","DPETECOP","DPETSPEP"]

for entry in district_map["features"]:
    for stat in pop_stats:
        entry["properties"][stat] = district.loc[entry["id"]][stat] # NaN (numpy.float64) when empty


In [26]:
district_map["features"][900]["properties"] # ["DISTRICT_N"]

{'DISTNAME': 'RANDOLPH FIELD ISD',
 'DISTRICT': '015-906',
 'DISTRICT_1': 15906,
 'DISTRICT_C': '015906',
 'DISTRICT_N': 15906,
 'DPETALLC': 1063,
 'DPETASIP': nan,
 'DPETBLAP': 23,
 'DPETECOP': 14.6,
 'DPETHISP': 15,
 'DPETINDP': nan,
 'DPETPCIP': nan,
 'DPETSPEP': 11,
 'DPETTWOP': nan,
 'DPETWHIP': 58,
 'NAME2': 'Randolph Field',
 'OBJECTID': 901,
 'OBJECTID_1': 901,
 'OBJECTID_2': 84,
 'REGION': 20}

In [ ]:
district

In [14]:
for row in district[:5]:
    print(row)

DISTRICT
DISTNAME
REGION
DPETALLC
DPETBLAP
DPETHISP
DPETWHIP
DPETINDP
DPETASIP
DPETPCIP
DPETTWOP
DPETECOP
DPETSPEP
AMERICAN INDIAN OR ALASKA NAT
ASIAN
BLACK OR AFRICAN AMERICAN
Economic Disadvantage
HISPANIC/LATINO
NATIVE HAWAIIAN/OTHER PACIFIC
Special Education
TWO OR MORE RACES
WHITE


In [93]:
apple = pd.concat(df_list, axis=1)

In [15]:
help(pd.read_csv)

Help on function read_csv in module pandas.io.parsers:

read_csv(filepath_or_buffer, sep=',', delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, iterator=False, chunksize=None, compression='infer', thousands=None, decimal=b'.', lineterminator=None, quotechar='"', quoting=0, escapechar=None, comment=None, encoding=None, dialect=None, tupleize_cols=None, error_bad_lines=True, warn_bad_lines=True, skipfooter=0, skip_footer=0, doublequote=True, delim_whitespace=False, as_recarray=None, compact_ints=None, use_unsigned=None, low_memory=True, buffer_lines=None, memory_map=False, float_precision=None)


In [94]:
pd.options.display.max_columns = 999
apple[:10]

REGION                      DISTNAME  \
DISTRICT HEADING AGGREGATION LEVEL                                         
31802    A01     DISTRICT SUMMARY      1.0  EAGLE ACADEMY OF BROWNSVILLE   
31803    A01     DISTRICT SUMMARY      NaN                           NaN   
         A02     DISTRICT SUMMARY      NaN                           NaN   
         A03     DISTRICT SUMMARY      NaN                           NaN   
         B10     DISTRICT SUMMARY      NaN                           NaN   
         B12     DISTRICT SUMMARY      NaN                           NaN   
         B13     DISTRICT SUMMARY      NaN                           NaN   
         B15     DISTRICT SUMMARY      NaN                           NaN   
         C16     DISTRICT SUMMARY      NaN                           NaN   
         C17     DISTRICT SUMMARY      NaN                           NaN   

                                            SECTION  \
DISTRICT HEADING AGGREGATION LEVEL                    
31802    A01     DISTRICT SUMMARY   A-PARTICIPATION   
31803    A01     DISTRICT SUMMARY               NaN   
         A02     DISTRICT SUMMARY               NaN   
         A03     DISTRICT SUMMARY               NaN   
         B10     DISTRICT SUMMARY               NaN   
         B12     DISTRICT SUMMARY               NaN   
         B13     DISTRICT SUMMARY               NaN   
         B15     DISTRICT SUMMARY               NaN   
         C16     DISTRICT SUMMARY               NaN   
         C17     DISTRICT SUMMARY               NaN   

                                                               HEADING NAME  \
DISTRICT HEADING AGGREGATION LEVEL                                            
31802    A01     DISTRICT SUMMARY   DISTRICT CUMULATIVE YEAR END ENROLLMENT   
31803    A01     DISTRICT SUMMARY                                       NaN   
         A02     DISTRICT SUMMARY                                       NaN   
         A03     DISTRICT SUMMARY                                       NaN   
         B10     DISTRICT SUMMARY                                       NaN   
         B12     DISTRICT SUMMARY                                       NaN   
         B13     DISTRICT SUMMARY                                       NaN   
         B15     DISTRICT SUMMARY                                       NaN   
         C16     DISTRICT SUMMARY                                       NaN   
         C17     DISTRICT SUMMARY                                       NaN   

                                     YR06  REGION DISTNAME SECTION  \
DISTRICT HEADING AGGREGATION LEVEL                                   
31802    A01     DISTRICT SUMMARY   285.0     NaN      NaN     NaN   
31803    A01     DISTRICT SUMMARY     NaN     NaN      NaN     NaN   
         A02     DISTRICT SUMMARY     NaN     NaN      NaN     NaN   
         A03     DISTRICT SUMMARY     NaN     NaN      NaN     NaN   
         B10     DISTRICT SUMMARY     NaN     NaN      NaN     NaN   
         B12     DISTRICT SUMMARY     NaN     NaN      NaN     NaN   
         B13     DISTRICT SUMMARY     NaN     NaN      NaN     NaN   
         B15     DISTRICT SUMMARY     NaN     NaN      NaN     NaN   
         C16     DISTRICT SUMMARY     NaN     NaN      NaN     NaN   
         C17     DISTRICT SUMMARY     NaN     NaN      NaN     NaN   

                                   HEADING NAME  YR07  REGION DISTNAME  \
DISTRICT HEADING AGGREGATION LEVEL                                       
31802    A01     DISTRICT SUMMARY           NaN   NaN     NaN      NaN   
31803    A01     DISTRICT SUMMARY           NaN   NaN     NaN      NaN   
         A02     DISTRICT SUMMARY           NaN   NaN     NaN      NaN   
         A03     DISTRICT SUMMARY           NaN   NaN     NaN      NaN   
         B10     DISTRICT SUMMARY           NaN   NaN     NaN      NaN   
         B12     DISTRICT SUMMARY           NaN   NaN     NaN      NaN   
         B13     DISTRICT SUMMARY           NaN   NaN     NaN      NaN   
      

In [61]:
df_list[0].index.get_level_values(0).unique()

Int64Index([ 31901, 108902,  31802, 240802, 108806, 108903, 108904, 240801,
             31903, 108905, 108807, 124901,  31905, 108912, 108914, 240901,
            245901,  31906, 245902, 108906, 108907, 108804, 108908, 108915,
            108801, 108909,  31909, 108910, 245903, 214901,  31911, 214903,
             31912, 214902, 245904,  31913,  31914, 108911,  31916, 108802,
            240903, 108916, 108808, 240904, 108913, 253901],
           dtype='int64', name='DISTRICT')

In [67]:
df_list[0].MultiIndex.is_unique()

AttributeError: 'DataFrame' object has no attribute 'MultiIndex'

In [69]:
df_list[0].describe()

,REGION,YR06
count,3308.0,3308.000000
mean,1.0,-251490.880593
std,0.0,434661.748009
min,1.0,-999999.000000
25%,1.0,-999999.000000
50%,1.0,30.000000
75%,1.0,238.000000
max,1.0,51437.000000


In [79]:
for x in df_list:
    print(x.describe())
    print(x[~x.index.duplicated(keep='last')].describe())

       REGION           YR06
count  3308.0    3308.000000
mean      1.0 -251490.880593
std       0.0  434661.748009
min       1.0 -999999.000000
25%       1.0 -999999.000000
50%       1.0      30.000000
75%       1.0     238.000000
max       1.0   51437.000000
       REGION           YR06
count  3308.0    3308.000000
mean      1.0 -251490.880593
std       0.0  434661.748009
min       1.0 -999999.000000
25%       1.0 -999999.000000
50%       1.0      30.000000
75%       1.0     238.000000
max       1.0   51437.000000
       REGION           YR07
count  3346.0    3346.000000
mean      1.0 -250395.330245
std       0.0  434062.636625
min       1.0 -999999.000000
25%       1.0 -999999.000000
50%       1.0      30.000000
75%       1.0     239.750000
max       1.0   51557.000000
       REGION           YR07
count  3346.0    3346.000000
mean      1.0 -250395.330245
std       0.0  434062.636625
min       1.0 -999999.000000
25%       1.0 -999999.000000
50%       1.0      30.000000
75%       1.0 

In [ ]:
"""df_list = [pd.read_csv(applePath.format(str(n).zfill(2),str(year)[-2:]), index_col=["DISTRICT","HEADING"])\
           for n in range(1,21) for year in years]"""

In [62]:
result = pd.concat(df_list, axis=1)

Exception: cannot handle a non-unique multi-index!

In [40]:
result[1010:1020]

,AGGREGATION LEVEL,REGION,DISTNAME,DISTRICT,SECTION,HEADING,HEADING NAME,YR06,AGGREGATION LEVEL,REGION,DISTNAME,DISTRICT,SECTION,HEADING,HEADING NAME,YR07,AGGREGATION LEVEL,REGION,DISTNAME,DISTRICT,SECTION,HEADING,HEADING NAME,YR08,AGGREGATION LEVEL,REGION,DISTNAME,DISTRICT,SECTION,HEADING,HEADING NAME,YR09,AGGREGATION LEVEL,REGION,DISTNAME,DISTRICT,SECTION,HEADING,HEADING NAME,YR10,AGGREGATION LEVEL,REGION,DISTNAME,DISTRICT,SECTION,HEADING,HEADING NAME,YR11,AGGREGATION LEVEL,REGION,DISTNAME,DISTRICT,SECTION,HEADING,HEADING NAME,YR12,AGGREGATION LEVEL,REGION,DISTNAME,DISTRICT,SECTION,HEADING,HEADING NAME,YR13,AGGREGATION LEVEL,REGION,DISTNAME,DISTRICT,SECTION,HEADING,HEADING NAME,YR14,AGGREGATION LEVEL,REGION,DISTNAME,DISTRICT,SECTION,HEADING,HEADING NAME,YR15,AGGREGATION LEVEL,REGION,DISTNAME,DISTRICT,SECTION,HEADING,HEADING NAME,YR16
1010,DISTRICT SUMMARY,1.0,LA VILLA ISD,108914.0,W-REASON INCIDENT COUNTS,G38,41-FIGHTING MUTUAL COMBAT,11.0,DISTRICT SUMMARY,1.0,LA VILLA ISD,108914.0,G-IN SCHOOL SUSPENSIONS,C23,HISPANIC,168.0,DISTRICT SUMMARY,1,LA JOYA ISD,108912,W-REASON INCIDENT COUNTS,G52,"52-ILLIGAL KNIFE,CLUB,OR WEAPON",-99999999,DISTRICT SUMMARY,1.0,LA VILLA ISD,108914.0,O-ECO. DISADV. DAEP PLACEMENTS,E10,ECO. DISAD. DAEP PLACEMENTS,-99999999.0,DISTRICT SUMMARY,1.0,LA JOYA ISD,108912.0,W-REASON INCIDENT COUNTS,G42,42-TRUANCY - PARENT CONTRIBUTE TO,66.0,DISTRICT SUMMARY,1.0,LA JOYA ISD,108912.0,X-DISCIPLINE ACTION COUNTS,H05,05-OUT-OF-SCHOOL SUSPENSION,7575.0,DISTRICT SUMMARY,1.0,LA JOYA ISD,108912.0,W-REASON INCIDENT COUNTS,G42,42-TRUANCY - PARENT CONTRIBUTE TO,22.0,DISTRICT SUMMARY,1.0,LAREDO ISD,240901.0,B-DISCIPLINE DATA TRENDS,B10,COUNT OF STUDENTS SUSPENDED IN SCHOOL,2464.0,DISTRICT SUMMARY,1.0,LAREDO ISD,240901.0,S-AT RISK EXPULSIONS,F05,AT RISK STUDENTS EXPELLED,69.0,DISTRICT SUMMARY,1.0,LAREDO ISD,240901.0,K-SPEC. ED. OUT OF SCHOOL SUS.,D12,NON SPEC. ED. OUT OF SCHOOL SUSPEN,1308.0,DISTRICT SUMMARY,1.0,LAREDO ISD,240901.0,W-REASON INCIDENT COUNTS,G59,59-SERIOUS MISBEHAV WHILE IN DAEP,43.0
1011,DISTRICT SUMMARY,1.0,LA VILLA ISD,108914.0,X-DISCIPLINE ACTION COUNTS,H05,05-OUT OF SCHOOL SUSPENSION,202.0,DISTRICT SUMMARY,1.0,LA VILLA ISD,108914.0,J-SPEC. ED. DAEP PLACEMENTS,D07,SPEC. ED. STUDENTS IN DAEP PLACEMENT,-999999.0,DISTRICT SUMMARY,1,LA JOYA ISD,108912,W-REASON INCIDENT COUNTS,G54,"54-FELONY MARIHUANA,CTRL SUBS",-99999999,DISTRICT SUMMARY,1.0,LA VILLA ISD,108914.0,O-ECO. DISADV. DAEP PLACEMENTS,E11,NON ECO. DISAD. DAEP PLACEMENTS,-99999999.0,DISTRICT SUMMARY,1.0,LA JOYA ISD,108912.0,W-REASON INCIDENT COUNTS,G43,43-TRUANCY - 3 UNEXCUSED ABSENCES,69.0,DISTRICT SUMMARY,1.0,LA JOYA ISD,108912.0,X-DISCIPLINE ACTION COUNTS,H06,06-IN-SCHOOL SUSPENSION,8171.0,DISTRICT SUMMARY,1.0,LA JOYA ISD,108912.0,W-REASON INCIDENT COUNTS,G43,43-TRUANCY - 3 UNEXCUSED ABSENCES,35.0,DISTRICT SUMMARY,1.0,LAREDO ISD,240901.0,B-DISCIPLINE DATA TRENDS,B13,STUDENTS SUSPENDED OUT OF SCHOOL,682.0,DISTRICT SUMMARY,1.0,LAREDO ISD,240901.0,S-AT RISK EXPULSIONS,F06,AT RISK EXPULSIONS,72.0,DISTRICT SUMMARY,1.0,LAREDO ISD,240901.0,L-SPEC. ED. IN SCHOOL SUS.,D13,SPEC. ED. STUDENTS IN SCHOOL SUSPEN,232.0,DISTRICT SUMMARY,1.0,LAREDO ISD,240901.0,X-DISCIPLINE ACTION COUNTS,H02,02-EXPULSION TO A JJAEP,83.0
1012,DISTRICT SUMMARY,1.0,LA VILLA ISD,108914.0,X-DISCIPLINE ACTION COUNTS,H06,06-IN SCHOOL SUSPENSION,220.0,DISTRICT SUMMARY,1.0,LA VILLA ISD,108914.0,J-SPEC. ED. DAEP PLACEMENTS,D08,SPEC. ED. DAEP PLACEMENTS,-999999.0,DISTRICT SUMMARY,1,LA JOYA ISD,108912,X-DISCIPLINE ACTION COUNTS,H02,02-EXPULSION TO A JJAEP,54,DISTRICT SUMMARY,1.0,LA VILLA ISD,108914.0,O-ECO. DISADV. DAEP PLACEMENTS,E12,UNKNOWN ECO. STATUS DAEP PLACEMENTS,-99999999.0,DISTRICT SUMMARY,1.0,LA JOYA ISD,108912.0,W-REASON INCIDENT COUNTS,G44,44-TRUANCY - 10 UNEXCUSED ABSENCE,616.0,DISTRICT SUMMARY,1.0,LA JOYA ISD,108912.0,X-DISCIPLINE ACTION COUNTS,H07,07-PLACEMENT IN ON/OFF CAMP DAEP,595.0,DISTRICT SUMMARY,1.0,LA JOYA ISD,108912.0,W-REASON INCIDENT COUNTS,G44,44-TRUANCY - 10 UNEXCU

In [31]:
pd.options.display.max_rows = 999
result.query('DISTRICT == "240903"')

AGGREGATION LEVEL    DISTNAME  \
DISTRICT HEADING                                 
240903   A01      DISTRICT SUMMARY  UNITED ISD   
         A02      DISTRICT SUMMARY  UNITED ISD   
         A03      DISTRICT SUMMARY  UNITED ISD   
         B01      DISTRICT SUMMARY  UNITED ISD   
         B02      DISTRICT SUMMARY  UNITED ISD   
         B03      DISTRICT SUMMARY  UNITED ISD   
         B04      DISTRICT SUMMARY  UNITED ISD   
         B05      DISTRICT SUMMARY  UNITED ISD   
         B06      DISTRICT SUMMARY  UNITED ISD   
         B07      DISTRICT SUMMARY  UNITED ISD   
         B08      DISTRICT SUMMARY  UNITED ISD   
         B09      DISTRICT SUMMARY  UNITED ISD   
         B10      DISTRICT SUMMARY  UNITED ISD   
         B11      DISTRICT SUMMARY  UNITED ISD   
         B12      DISTRICT SUMMARY  UNITED ISD   
         B13      DISTRICT SUMMARY  UNITED ISD   
         B14      DISTRICT SUMMARY  UNITED ISD   
         B15      DISTRICT SUMMARY  UNITED ISD   
         C03      DISTRICT SUMMARY  UNITED ISD   
         C05      DISTRICT SUMMARY  UNITED ISD   
         C08      DISTRICT SUMMARY  UNITED ISD   
         C10      DISTRICT SUMMARY  UNITED ISD   
         C13      DISTRICT SUMMARY  UNITED ISD   
         C15      DISTRICT SUMMARY  UNITED ISD   
         C16      DISTRICT SUMMARY  UNITED ISD   
         C18      DISTRICT SUMMARY  UNITED ISD   
         C20      DISTRICT SUMMARY  UNITED ISD   
         C21      DISTRICT SUMMARY  UNITED ISD   
         C22      DISTRICT SUMMARY  UNITED ISD   
         C23      DISTRICT SUMMARY  UNITED ISD   
         C25      DISTRICT SUMMARY  UNITED ISD   
         D01      DISTRICT SUMMARY  UNITED ISD   
         D02      DISTRICT SUMMARY  UNITED ISD   
         D03      DISTRICT SUMMARY  UNITED ISD   
         D04      DISTRICT SUMMARY  UNITED ISD   
         D05      DISTRICT SUMMARY  UNITED ISD   
         D06      DISTRICT SUMMARY  UNITED ISD   
         D07      DISTRICT SUMMARY  UNITED ISD   
         D08      DISTRICT SUMMARY  UNITED ISD   
         D09      DISTRICT SUMMARY  UNITED ISD   
         D10      DISTRICT SUMMARY  UNITED ISD   
         D11      DISTRICT SUMMARY  UNITED ISD   
         D12      DISTRICT SUMMARY  UNITED ISD   
         D13      DISTRICT SUMMARY  UNITED ISD   
         D14      DISTRICT SUMMARY  UNITED ISD   
         D15      DISTRICT SUMMARY  UNITED ISD   
         E01      DISTRICT SUMMARY  UNITED ISD   
         E02      DISTRICT SUMMARY  UNITED ISD   
         E03      DISTRICT SUMMARY  UNITED ISD   
         E04      DISTRICT SUMMARY  UNITED ISD   
         E05      DISTRICT SUMMARY  UNITED ISD   
         E06      DISTRICT SUMMARY  UNITED ISD   
         E07      DISTRICT SUMMARY  UNITED ISD   
         E08      DISTRICT SUMMARY  UNITED ISD   
         E09      DISTRICT SUMMARY  UNITED ISD   
         E10      DISTRICT SUMMARY  UNITED ISD   
         E11      DISTRICT SUMMARY  UNITED ISD   
         E12      DISTRICT SUMMARY  UNITED ISD   
         E13      DISTRICT SUMMARY  UNITED ISD   
         E14      DISTRICT SUMMARY  UNITED ISD   
         E15      DISTRICT SUMMARY  UNITED ISD   
         E16      DISTRICT SUMMARY  UNITED ISD   
         E17      DISTRICT SUMMARY  UNITED ISD   
         E18      DISTRICT SUMMARY  UNITED ISD   
         E19      DISTRICT SUMMARY  UNITED ISD   
         E20      DISTRICT SUMMARY  UNITED ISD   
         F01      DISTRICT SUMMARY  UNITED ISD   
         F02      DISTRICT SUMMARY  UNITED ISD   
         F03      DISTRICT SUMMARY  UNITED ISD   
         F04      DISTRICT SUMMARY  UNITED ISD   
         F05      DISTRICT SUMMARY  UNITED ISD   
         F06      DISTRICT SUMMARY  UNITED ISD   
         F07      DISTRICT SUMMARY  UNITED ISD   
         F08      DISTRICT SUMMARY  UNITED ISD   
         F09      DISTRICT SUMMARY  UNITED ISD   
         F10      DISTRICT SUMMARY  UNITED ISD   
         F11      DISTRICT SUMMARY  UNITED ISD   
         F12      DISTRICT SUMMARY  UNITED ISD   
         F13     

In [29]:
result[3000:3010]

AGGREGATION LEVEL         DISTNAME  \
DISTRICT HEADING                                      
240903   H17      DISTRICT SUMMARY       UNITED ISD   
         H18      DISTRICT SUMMARY       UNITED ISD   
         H19      DISTRICT SUMMARY       UNITED ISD   
108916   A01      DISTRICT SUMMARY  VALLEY VIEW ISD   
         A02      DISTRICT SUMMARY  VALLEY VIEW ISD   
         A03      DISTRICT SUMMARY  VALLEY VIEW ISD   
         B07      DISTRICT SUMMARY  VALLEY VIEW ISD   
         B08      DISTRICT SUMMARY  VALLEY VIEW ISD   
         B09      DISTRICT SUMMARY  VALLEY VIEW ISD   
         B10      DISTRICT SUMMARY  VALLEY VIEW ISD   

                                             HEADING NAME  REGION  \
DISTRICT HEADING                                                    
240903   H17            17-TRUANCY/CHARGES FILED W/O FINE       1   
         H18            25-PART DAY OUT OF SCHOOL SUSPEND       1   
         H19                26-PART DAY IN SCHOOL SUSPEND       1   
108916   A01      DISTRICT CUMULATIVE YEAR END ENROLLMENT       1   
         A02               DISTRICT DISCIPLINE POPULATION       1   
         A03             DISTRICT DISCIPLINE RECORD COUNT       1   
         B07          COUNT OF STUDENTS REMOVED TO A DAEP       1   
         B08                      MANDATORY DAEP REMOVALS       1   
         B09                  DISCRETIONARY DAEP REMOVALS       1   
         B10        COUNT OF STUDENTS SUSPENDED IN SCHOOL       1   

                                     SECTION      YR06  YR07  YR08  YR09  \
DISTRICT HEADING                                                           
240903   H17      X-DISCIPLINE ACTION COUNTS     225.0   NaN   NaN   NaN   
         H18      X-DISCIPLINE ACTION COUNTS      56.0   NaN   NaN   NaN   
         H19      X-DISCIPLINE ACTION COUNTS     901.0   NaN   NaN   NaN   
108916   A01                 A-PARTICIPATION    4348.0   NaN   NaN   NaN   
         A02                 A-PARTICIPATION     362.0   NaN   NaN   NaN   
         A03                 A-PARTICIPATION     591.0   NaN   NaN   NaN   
         B07        B-DISCIPLINE DATA TRENDS      12.0   NaN   NaN   NaN   
         B08        B-DISCIPLINE DATA TRENDS       8.0   NaN   NaN   NaN   
         B09        B-DISCIPLINE DATA TRENDS -999999.0   NaN   NaN   NaN   
         B10        B-DISCIPLINE DATA TRENDS     320.0   NaN   NaN   NaN   

                  YR10  YR11  YR12  YR13  YR14  YR15  YR16  
DISTRICT HEADING                                            
240903   H17       NaN   NaN   NaN   NaN   NaN   NaN   NaN  
         H18       NaN   NaN   NaN   NaN   NaN   NaN   NaN  
         H19       NaN   NaN   NaN   NaN   NaN   NaN   NaN  
108916   A01       NaN   NaN   NaN   NaN   NaN   NaN   NaN  
         A02       NaN   NaN   NaN   NaN   NaN   NaN   NaN  
         A03       NaN   NaN   NaN   NaN   NaN   NaN   NaN  
         B07       NaN   NaN   NaN   NaN   NaN   NaN   NaN  
         B08       NaN   NaN   NaN   NaN   NaN   NaN   NaN  
         B09       NaN   NaN   NaN   NaN   NaN   NaN   NaN  
         B10       NaN   NaN   NaN   NaN   NaN   NaN   NaN